Vectorizar raster (labels -> GeoPackage)
**JOAN IMPLEMENTACION**

Métrica MAE (básica) entre vectores (distance-based)
**JOAN IMPLEMENTACION**

# agrergo las etiquetas
### primero voy a recortas los rasters del inta

In [ ]:
import pandas as pd, pickle, numpy as np

df = pd.read_csv('C:\Users\lean3\Documents\ade_final\data\dataframe_region_stats.csv')      # CSV
ndvi_list = pickle.load(open('C:\Users\lean3\Documents\ade_final\data\ndvi_list.pkl','rb')) # PKL
labels_ws = np.load('C:\Users\lean3\Documents\ade_final\data\labels_ws.npy')                # NPY


[array([[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],
       shape=(4998, 4979), dtype=float32),
 array([[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],
       shape=(4998, 4979), dtype=float32),
 array([[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],
       shape=(4998, 4979), dtype=float32),
 array([[nan, nan, 

In [ ]:
import rasterio
import numpy as np
from rasterio.warp import reproject, Resampling

r1_path = r"C:\Users\eugea\Downloads\MNC_invierno2023.tif"
r2_path = r"C:\Users\eugea\Downloads\Sentinel2_TresArroyos\S2_RGB_NIR_NDVI_NDWI_2024-07.tif"

# Abrir referencia
with rasterio.open(r2_path) as r2:
    ref_profile = r2.profile
    ref_transform = r2.transform
    ref_crs = r2.crs
    ref_shape = (r2.height, r2.width)

# Abrir raster fuente
with rasterio.open(r1_path) as r1:

    # Preparamos resultado vacío
    r1_data_aligned = np.zeros(ref_shape, dtype=r1.meta["dtype"])

    # Reproyectar
    reproject(
        source=rasterio.band(r1, 1),
        destination=r1_data_aligned,
        src_transform=r1.transform,
        src_crs=r1.crs,
        dst_transform=ref_transform,
        dst_crs=ref_crs,
        resampling=Resampling.nearest
    )


out_path = "recorte_invierno.tif"
profile_out = ref_profile.copy()

profile_out.update(
    dtype="float32",
    nodata=-9999,
    count=1
)

r1_data_aligned = r1_data_aligned.astype("float32")

# Guardar
with rasterio.open(out_path, "w", **profile_out) as dst:
    dst.write(r1_data_aligned, 1)

print("✅ Listo:", out_path)

In [ ]:
from funciones import guardar_GTiff

out_path = r"C:\Users\eugea\Downloads\recorte_invierno_GTiff.tif"

guardar_GTiff(
    fn=out_path,
    crs=ref_crs,
    transform=ref_transform,
    mat=r1_data_aligned.astype(np.float32),
    nodata=-9999,
    bandnames=["MNC_2024"]
)

print("✅ Guardado en:", out_path)

hay que hacer lo mismo para el de verano

### elijo los campos

In [ ]:
with rasterio.open(r"C:\Users\eugea\Downloads\recorte_verano_GTiff.tif") as src:
    ver = src.read(1)

CAMPOS_VERANO = [11, 10, 12, 13, 15, 14, 17, 16, 19, 18, 26, 28, 30, 21]
binary_ver = np.isin(ver, CAMPOS_VERANO).astype(np.uint8)

In [ ]:
with rasterio.open(r"C:\Users\eugea\Downloads\recorte_invierno_GTiff.tif") as src:
    inv = src.read(1)


CAMPOS_INVIERNO = [16, 19, 24, 6, 26, 17, 18]
binary_inv = np.isin(inv, CAMPOS_INVIERNO).astype(np.uint8)

In [ ]:
binary_combined = np.logical_or(binary_inv == 1, binary_ver == 1).astype(np.uint8)

combino los campos de verano e invierno

In [ ]:
profile = src.profile.copy()
profile.update(
    dtype=rasterio.uint8,
    count=1,
    nodata=0
)

with rasterio.open("INTA_campo_binario.tif", "w", **profile) as dst:
    dst.write(binary_combined.astype(np.uint8), 1)

### paso 1 verifico shapes

In [ ]:
labels_ws.shape, binary_combined.shape

### paso 2 obtengo los id segmentos

In [ ]:
import numpy as np

segment_ids = np.unique(labels_ws)
print("Número de segmentos:", len(segment_ids))

### PASO 3 — Calcular etiqueta campo/no-campo por segmento

Si ≥ 70% de los píxeles del segmento son campo → segmento = campo
Sino → no-campo

In [ ]:
# diccionario: id_segmento → clase (0/1)
segment_class = {}

for seg_id in segment_ids:
    if seg_id == 0:
        continue   # 0 puede ser background, opcional
    mask = labels_ws == seg_id
    vals = binary_combined[mask]      # pixeles INTA dentro del segmento
    pct = vals.mean()                 # % de pixeles = 1
    segment_class[seg_id] = 1 if pct >= 0.7 else 0

### paso 4 creo rasters de etiquetas

In [ ]:
segment_label_raster = np.zeros_like(labels_ws, dtype=np.uint8)

for seg_id, cls in segment_class.items():
    segment_label_raster[labels_ws == seg_id] = cls

Ahora segment_label_raster es un raster 2D:

1 = campo

0 = no campo

### paso 5 guardo en goetiff

In [ ]:
with rasterio.open(r"C:\Users\eugea\Downloads\recorte_invierno_GTiff.tif") as src:
    crs = src.crs
    transform = src.transform

out_path = r"C:\Users\eugea\Downloads"
guardar_GTiff(
    fn=out_path + "\segmentos_etiquetados2.tif",
    crs=crs,
    transform=transform,
    mat=segment_label_raster,
    meta={"dtype": "uint8"},
    nodata=0,
    bandnames=["campo"]
)

Para vizualizacion rápida

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(8,8))
plt.imshow(segment_label_raster, cmap="RdYlGn")
plt.colorbar()
plt.title("Segmentos Etiquetados: 1=Campo, 0=No Campo")
plt.axis('off')